In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np

import getpass
user = getpass.getuser()

In [ ]:
from PIL import Image

In [ ]:
from omni.isaac.kit import SimulationApp

# "/home/yizhou/Research/OpenAnyDrawer/scene0.usd" # 
usd_path = f"omniverse://localhost/Users/{user}/scene4.usd"


simulation_app = SimulationApp({"headless": True, "open_usd": usd_path,  "livesync_usd": usd_path}) 


In [ ]:
# world
from omni.isaac.core import World
world = World()

In [ ]:
# reset scene
import omni
mobility_prim = world.scene.stage.GetPrimAtPath("/World/Game/mobility")
if mobility_prim:
    omni.kit.commands.execute("DeletePrims", paths=["/World/Game/mobility"])
    
# hide robot
# hand_prim = world.scene.stage.GetPrimAtPath("/World/allegro")
# hand_prim.GetAttribute('visibility').Set('invisible')

In [ ]:
from open_env import OpenEnv

env = OpenEnv()

In [ ]:
from franka.gripper import GripperHandEnv

# env = HandEnv("/World/Hand/Bones/l_carpal_mid", "/World/Hand*/Bones/l_thumbSkeleton_grp/l_distalThumb_mid")
controller = GripperHandEnv("/World/Franka/panda_link8", "/World/AnchorXform")

In [ ]:
env.add_camera()
env.setup_viewport()

In [ ]:
# init
world.reset()
controller.start()
world.scene.add(controller.robots)

world.render()
env.get_image()

In [ ]:
world.render()
env.get_image()

In [ ]:
env.add_object(0, scale = 0.1)

In [ ]:
from omni.isaac.core.prims.xform_prim import XFormPrim
mobility_obj = XFormPrim("/World/Game/mobility")
mobility_obj_name = mobility_obj.name

world.scene.add(mobility_obj)

In [ ]:
from task.checker import TaskChecker
from task.instructor import SceneInstructor

In [ ]:
scene_instr = SceneInstructor()
scene_instr.analysis()

In [ ]:
scene_instr.is_obj_valid

In [ ]:
scene_instr.valid_handle_list

In [ ]:
handle_path_str = list(scene_instr.valid_handle_list.keys())[0]

handle_joint_type = scene_instr.valid_handle_list[handle_path_str]["joint_type"]
handle_joint = scene_instr.valid_handle_list[handle_path_str]["joint"]
handle_rel_direciton = scene_instr.valid_handle_list[handle_path_str]["relative_to_game_center"]
handle_direction = scene_instr.valid_handle_list[handle_path_str]["direction"]

In [ ]:
handle_path_str, handle_joint_type, handle_joint, handle_rel_direciton

In [ ]:
task_checker = TaskChecker("mobility", handle_joint, handle_joint_type, IS_RUNTIME=True)

# Deep Learning

In [ ]:
scene_instr.load_model()

In [ ]:
#image_array =env.get_image(return_array=True, world = world)
image_array =env.get_image(return_array=True, world = world)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(image_array[:,:,:3]) 

In [ ]:
scene_instr.predict_bounding_boxes(image_array[:,:,:3])

In [ ]:
scene_instr.pred_boxes

In [ ]:
v_desc = scene_instr.valid_handle_list[handle_path_str]["vertical_description"]
h_desc = scene_instr.valid_handle_list[handle_path_str]["horizontal_description"]

the_box = scene_instr.get_box_from_desc(v_desc, h_desc)

In [ ]:
the_box = [-0.24306170648464168, 0.9585027645051194, -0.2043927986348123, 0.9971716723549489]

In [ ]:
pred_handle_direction = "horizontal" if (the_box[2] - the_box[0]) > (the_box[3] - the_box[1]) else "vertical" 

In [ ]:
pred_handle_direction

In [ ]:
the_box[3] - the_box[1]

In [ ]:
pred_handle_direction

In [ ]:
graps_pos, grasp_rot = controller.calculate_grasp_location_from_pred_box(the_box, False)

# Solve

In [ ]:
world.reset()

In [ ]:
controller.xforms.set_world_poses(positions=np.array([[0,0,0]]), orientations = np.array([[1, 0, 0, 0]])) # WXYZ

for _ in range(60):
    world.step(render=True)

In [ ]:
# get grasp location, if handle is horizontal, gripper should be vertical
graps_pos, grasp_rot = controller.calculate_grasp_location(keyword = handle_path_str, 
                                                         verticle = handle_direction == "horizontal")

In [ ]:
graps_pos, grasp_rot

# Control

In [ ]:
graps_pos, grasp_rot

In [ ]:
graps_pos[...,0] -= 0.1

In [ ]:
controller.xforms.set_world_poses(graps_pos, grasp_rot)

for _ in range(200):
    world.step(render=True)

In [ ]:
graps_pos[...,0] += 0.1

In [ ]:
controller.xforms.set_world_poses(graps_pos, grasp_rot)

for _ in range(100):
    world.step(render=True)

In [ ]:
# close
pos = np.array([[0.0, 0.0]])
               
for _ in range(100):
    pos -= 0.01
    controller.robots.set_joint_position_targets(pos)
    pos = np.clip(pos, 0, 4)
    world.step(render=True)

In [ ]:
# pull out
for i in range(300):
    graps_pos[...,0] -= 0.001
    pos += np.sqrt(i) * 1e-4
    # print(pos)
    controller.xforms.set_world_poses(graps_pos, grasp_rot)
    controller.robots.set_joint_position_targets(pos)
    
    
    pos = np.clip(pos, 0, 4)
    world.step(render=True)

In [ ]:
# check task
task_checker.joint_checker.compute_percentage()

# Reset

In [ ]:
world.reset()
controller.xforms.set_world_poses(positions=np.array([[0,0,0]]), orientations = np.array([[1, 0, 0, 0]])) # WXYZ

for _ in range(30):
    world.step()

In [ ]:
world.scene.remove_object(mobility_obj_name)